In [1]:
! pip install torchsummary

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
                 
        self.layer1 = nn.Sequential(
        			nn.Conv2d( 3, 16, kernel_size=3, stride=2, padding=0 ),
                    nn.BatchNorm2d(16),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2))
                    
        self.layer2 = nn.Sequential(
        			nn.Conv2d( 16, 32, kernel_size=3, stride=2, padding=0),
                    nn.BatchNorm2d(32),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2))
                    
        self.layer3 = nn.Sequential(
        			nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
                    nn.BatchNorm2d(64),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=2, stride=2))
                    
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3*3*64, 1000)
        self.fc2 = nn.Linear(1000,1)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [4]:
# 모델 초기화
model = TheModelClass().cuda()

# optimizer 초기화
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [5]:
#모델 파라미터 출력

print("Model's state_dict:")
for param_tensor in model.state_dict():
	print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
layer1.0.weight 	 torch.Size([16, 3, 3, 3])
layer1.0.bias 	 torch.Size([16])
layer1.1.weight 	 torch.Size([16])
layer1.1.bias 	 torch.Size([16])
layer1.1.running_mean 	 torch.Size([16])
layer1.1.running_var 	 torch.Size([16])
layer1.1.num_batches_tracked 	 torch.Size([])
layer2.0.weight 	 torch.Size([32, 16, 3, 3])
layer2.0.bias 	 torch.Size([32])
layer2.1.weight 	 torch.Size([32])
layer2.1.bias 	 torch.Size([32])
layer2.1.running_mean 	 torch.Size([32])
layer2.1.running_var 	 torch.Size([32])
layer2.1.num_batches_tracked 	 torch.Size([])
layer3.0.weight 	 torch.Size([64, 32, 3, 3])
layer3.0.bias 	 torch.Size([64])
layer3.1.weight 	 torch.Size([64])
layer3.1.bias 	 torch.Size([64])
layer3.1.running_mean 	 torch.Size([64])
layer3.1.running_var 	 torch.Size([64])
layer3.1.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([1000, 576])
fc1.bias 	 torch.Size([1000])
fc2.weight 	 torch.Size([1, 1000])
fc2.bias 	 torch.Size([1])


In [6]:
from torchsummary import summary
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 111, 111]             448
       BatchNorm2d-2         [-1, 16, 111, 111]              32
              ReLU-3         [-1, 16, 111, 111]               0
         MaxPool2d-4           [-1, 16, 55, 55]               0
            Conv2d-5           [-1, 32, 27, 27]           4,640
       BatchNorm2d-6           [-1, 32, 27, 27]              64
              ReLU-7           [-1, 32, 27, 27]               0
         MaxPool2d-8           [-1, 32, 13, 13]               0
            Conv2d-9             [-1, 64, 6, 6]          18,496
      BatchNorm2d-10             [-1, 64, 6, 6]             128
             ReLU-11             [-1, 64, 6, 6]               0
        MaxPool2d-12             [-1, 64, 3, 3]               0
          Dropout-13                  [-1, 576]               0
           Linear-14                 [-

In [7]:
MODEL_PATH = "saved"

if not os.path.exists(MODEL_PATH):
	os.makedirs(MODEL_PATH)

    # 모델의 파라미터 저장 
torch.save(model.state_dict(),
		   os.path.join(MODEL_PATH, "model.pt"))

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = TheModelClass()
# 모델 불러오기 -> 반드시 같은 아키텍처를 가져와야만 함 
new_model.load_state_dict(torch.load(os.path.join(MODEL_PATH,"model.pt")))


<All keys matched successfully>

In [9]:
# 모델 전체를 저장 (pickle 이용)

torch.save(model, os.path.join(MODEL_PATH, "model_pickle.pt"))
model = torch.load(os.path.join(MODEL_PATH, "model_pickle.pt"))
model.eval()

TheModelClass(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=576, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, b

### CheckPoints
학습의 중간 결과를 저장해 최선의 결과를 선택 

epoch, loss, metric 값을 지속적으로 확인 저장

In [10]:
import zipfile
import shutil 

filename = "kagglecatsanddogs_5340.zip"
with zipfile.ZipFile(filename, 'r') as zip_ref:
	zip_ref.extractall()
    
import shutil
shutil.move('PetImages', 'data')

import os 
from os import walk

mypath = "data"

f_path = []
for (dirpath, dirnames, filenames) in walk(mypath):
	f_path.extend([os.path.join(dirpath, filename) for filename in filenames])
    
from PIL import Image

# 유효하지 않은 파일들은 삭제 처리
for f_name in f_path:
    try:
        Image.open(f_name)
    except Exception as e:
        print(e)
        os.remove(f_name)

Error: Destination path 'data\PetImages' already exists

In [11]:
pwd

'C:\\Users\\asy10\\ML\\DNN\\basic_torch'

In [28]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch 

dataset = datasets.ImageFolder(root="./data/PetImages",
				transform=transforms.Compose([
                transforms.Resize(244),
                transforms.CenterCrop(244),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5],
        				 std = [0.5, 0.5, 0.5])
                ]))
                
dataloader = torch.utils.data.DataLoader(dataset,batch_size=8,shuffle=True,num_workers=8)

In [29]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc*100)
    
    return acc

In [30]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.1

In [31]:
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [32]:
for e in range( 1, EPOCHS + 1 ):
    epoch_loss = 0
    epoch_acc =0 
    # data 를 GPU 로 전달해줌 
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    # Check point 를 저장한다. 
    torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),# 모델의 파라미터 저장 
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        }, f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")
    print(f'Epoch {e+0:03}: | Loss {epoch_loss/len(dataloader):.5f} | Acc : {epoch_acc/len(dataloader):.3f}')

Epoch 001: | Loss 0.84544 | Acc : 49.795
Epoch 002: | Loss 0.85836 | Acc : 50.386
Epoch 003: | Loss 0.86080 | Acc : 50.347
Epoch 004: | Loss 0.85563 | Acc : 50.037
Epoch 005: | Loss 0.84687 | Acc : 50.131
Epoch 006: | Loss 0.88031 | Acc : 49.939
Epoch 007: | Loss 0.85371 | Acc : 49.816
Epoch 008: | Loss 0.83743 | Acc : 50.083
Epoch 009: | Loss 0.84291 | Acc : 49.828
Epoch 010: | Loss 0.84589 | Acc : 50.033
Epoch 011: | Loss 0.84697 | Acc : 49.832
Epoch 012: | Loss 0.85747 | Acc : 49.812
Epoch 013: | Loss 0.85328 | Acc : 49.745
Epoch 014: | Loss 0.86010 | Acc : 49.730
Epoch 015: | Loss 0.83031 | Acc : 49.862
Epoch 016: | Loss 0.87708 | Acc : 50.385
Epoch 017: | Loss 0.81394 | Acc : 50.158
Epoch 018: | Loss 0.86267 | Acc : 49.496
Epoch 019: | Loss 0.84285 | Acc : 50.022
Epoch 020: | Loss 0.84047 | Acc : 50.180
Epoch 021: | Loss 0.87165 | Acc : 49.263
Epoch 022: | Loss 0.84765 | Acc : 49.863
Epoch 023: | Loss 0.85097 | Acc : 50.342
Epoch 024: | Loss 0.85121 | Acc : 50.583
Epoch 025: | Los

## Pre-trained Model

- 사전에 학습된 모델을 의미하고 유사한 문제를 학습한 모델로부터 좋은 초기값을 얻기 위해서 사용

## Transfer Learning 

- 기존에 학습된 파라미터를 새로운 문제에 "전이" 시키도록 학습

- 다른 데이터셋으로 만든 모델을 현재 데이터에 적용

- 일반적으로 대용량 데이터셋으로 만들어진 모델의 성능 좋다

- backbone architecture가 잘 학습된 모델에서 일부분만 변경하여 학습을 수행


## Freezing

- pre-trained model을 활용시 모델의 일부분을 frozen 시킴 

### Pretrained Model Loading

In [34]:
import torch
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg = models.vgg16(pretrained=True).to(device)

print(vgg)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\asy10/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [41]:
from torchsummary import summary
summary(vgg, (3,244,244))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 244, 244]           1,792
              ReLU-2         [-1, 64, 244, 244]               0
            Conv2d-3         [-1, 64, 244, 244]          36,928
              ReLU-4         [-1, 64, 244, 244]               0
         MaxPool2d-5         [-1, 64, 122, 122]               0
            Conv2d-6        [-1, 128, 122, 122]          73,856
              ReLU-7        [-1, 128, 122, 122]               0
            Conv2d-8        [-1, 128, 122, 122]         147,584
              ReLU-9        [-1, 128, 122, 122]               0
        MaxPool2d-10          [-1, 128, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]         295,168
             ReLU-12          [-1, 256, 61, 61]               0
           Conv2d-13          [-1, 256, 61, 61]         590,080
             ReLU-14          [-1, 256,

In [42]:
for name, layer in vgg.named_modules():
    print(name, layer)

 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

In [43]:
vgg.fc = torch.nn.Linear(1000,1)
vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [40]:
#vgg.classifier._modules['6'] = torch.nn.Linear(4096,1) # 마지막 레이어 이름 변경해서 최종 output featrues = 1로 만들기 
#vgg.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [44]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch 

dataset = datasets.ImageFolder(root="./data/PetImages",
				transform=transforms.Compose([
                transforms.Resize(244),
                transforms.CenterCrop(244),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5],
        				 std = [0.5, 0.5, 0.5])
                ]))
                
dataloader = torch.utils.data.DataLoader(dataset,batch_size=8,shuffle=True,num_workers=8)

In [57]:
from torch import nn
from torchvision import models

class MyNewNet(nn.Module):
    def __init__(self):
        super(MyNewNet, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.linear_layers = nn.Linear(1000,1)
    
    def forward(self, x):
        x = self.vgg19(x)
        return self.linear_layers(x)

In [58]:
my_model = MyNewNet()
my_model = my_model.to(device)

In [59]:
print(my_model)

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [60]:
for param in my_model.parameters(): # frozeen
    param.requires_grad = False

In [61]:
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

In [62]:
my_model.eval()

MyNewNet(
  (vgg19): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [63]:
x = next(iter(dataloader))

In [64]:
x[0].shape # batch, 3 , 244 , 244

torch.Size([8, 3, 244, 244])

In [65]:
my_model(x[0].to(device))

tensor([[ 1.2265],
        [-0.5768],
        [-0.1067],
        [ 0.4275],
        [ 0.7496],
        [-0.5965],
        [ 0.0294],
        [-1.3035]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [66]:
x[1]

tensor([1, 0, 0, 0, 1, 0, 0, 0])

In [67]:
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]], device='cuda:0')

In [69]:
it = my_model.linear_layers.parameters() # frozeen 시키지 않음 /
next(it)[0][:10]

tensor([-0.0166, -0.0205, -0.0130, -0.0253, -0.0272, -0.0042,  0.0126,  0.0103,
        -0.0168,  0.0203], device='cuda:0', grad_fn=<SliceBackward0>)

In [71]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc*100)
    
    return acc

my_model = MyNewNet()
my_model = my_model.to(device)

for param in my_model.parameters(): #parameters frozeen
    param.requires_grad = False
    
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(my_model.parameters(), lr=LEARNING_RATE)

for e in range( 1, EPOCHS + 1 ):
    epoch_loss = 0
    epoch_acc =0 
    # data 를 GPU 로 전달해줌 
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()
        y_pred = my_model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    # Check point 를 저장한다. 
    torch.save({
        'epoch': e,
        'model_state_dict': my_model.state_dict(),# 모델의 파라미터 저장 
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        }, f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")
    print(f'Epoch {e+0:03}: | Loss {epoch_loss/len(dataloader):.5f} | Acc : {epoch_acc/len(dataloader):.3f}')